* 실행금지
<pre>
pip install ipympl
pip install nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib 

# backtrader
* ref : https://github.com/mementum/backtrader
* ref : https://www.backtrader.com/
* 차트 에러시 : https://github.com/mementum/backtrader/pull/418/commits/61f0878a4e3616b62d3ad035acafd0ff25da1dec

In [1]:
# ! pip install backtrader
# ! pip install ipympl

In [2]:
### # ! pip install IPython
import IPython

In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader as bt
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

# Create a subclass of Strategy to define the indicators and logic
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30  # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)         # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)         # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:          # not in the market
            if self.crossover > 0:     # if fast crosses slow to the upside
                self.buy()             # enter long
        elif self.crossover < 0:       # in the market & cross to the downside
            self.close()               # close long position
        
cerebro = bt.Cerebro()  
data  = pdr.get_data_yahoo('MSFT', '2018-01-01', '2021-12-31')
data1 = pdr.get_data_yahoo('AAPL', '2018-01-01', '2021-12-31')
data = bt.feeds.PandasData(dataname=data)
data1 = bt.feeds.PandasData(dataname=data1)
cerebro.adddata(data) 
cerebro.adddata(data1)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:

cerebro.broker.setcash(20200.0)
cerebro.addstrategy(SmaCross)  # Add the trading strategy
cerebro.run()  
cerebro.plot(iplot = False)
plt.show()

# stock_prices = pd.read_csv(path, index_col='datetime', parse_dates=True)
# Create a data feed
# data = bt.feeds.YahooFinanceData(dataname='MSFT',
#                                  fromdate=datetime(2011, 1, 1),
#                                  todate=datetime(2021, 12, 31))

# data1 = bt.feeds.YahooFinanceData(dataname='AAPL',
#                                  fromdate=datetime(2011, 1, 1),
#                                  todate=datetime(2021, 12, 31))


# Backtesting
* ref : https://github.com/kernc/backtesting.py

In [6]:
# ! pip install backtesting

In [7]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

C:\AI\pythonProject\venv\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Row(id='1450', ...)

In [8]:
bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
output

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                  68221.96986
Equity Peak [$]                   68991.21986
Return [%]                         582.219699
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   25.266427
Volatility (Ann.) [%]               38.383008
Sharpe Ratio                         0.658271
Sortino Ratio                        1.288779
Calmar Ratio                         0.763748
Max. Drawdown [%]                  -33.082172
Avg. Drawdown [%]                   -5.581506
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   94
Win Rate [%]                        54.255319
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

# self class

In [9]:
import pyupbit
import numpy as np

class MyBackTesting :
    def __init__(self, daily_ohlcv, start_cash) :
        self.daily_ohlcv = daily_ohlcv      # 일봉 데이터
        self.fee = 0.0011                   # 수수료 ( calculate_fee.xlsx 참고 )
        self.buy_signal = False             # 매수 신호
        
        self.start_cash   = start_cash      # 시작 자산
        self.current_cash = start_cash      # 현재 자산
        self.highest_cash = start_cash      # 자산 최고점
        self.lowest_cash  = start_cash      # 자산 최저점

        self.ror = 1                        # 수익률
        self.accumulated_ror = 1            # 누적 수익률
        self.mdd = 0                        # 최대 낙폭

        self.trade_count = 0                # 거래횟수
        self.win_count = 0                  # 승리횟수

    def my_execute(self) :
        K = 0.5
        
        # 변동폭 ( 고가 - 저가 )
        self.daily_ohlcv['range'] = self.daily_ohlcv['high'] - self.daily_ohlcv['low']
        # 목표매수가 ( 시가 + 변동폭 * K )
        self.daily_ohlcv['targetPrice'] = self.daily_ohlcv['open'] + self.daily_ohlcv['range'].shift(1) * K

        for idx, row in self.daily_ohlcv.iterrows() :
            # 매수 신호 확인
            # 목표가에 달성한 경우 목표가에 매수해 다음날 시가에 매도한 것으로 판단
            self.buy_signal = np.where(row['low'] <= row['targetPrice'] <= row['high'], True, False) 
            
            # 거래횟수 계산
            self.trade_count += 1 if self.buy_signal else 0
            # 수익률 계산
            self.ror = row['close'] / row['targetPrice'] - self.fee if self.buy_signal else 1 # 다음날 시가와 당일 종가의 시간차이가 거의 없으므로 종가로 계산
            # 승리 횟수 계산
            self.win_count += 1 if self.ror > 1 else 0
            # 누적 수익률 계산
            self.accumulated_ror *= self.ror
            # 현재 자산 갱신
            self.current_cash *= self.ror
            # 자산 최고점 갱신
            self.highest_cash = max(self.highest_cash, self.current_cash)
            # 자산 최저점 갱신
            self.lowest_cash = min(self.lowest_cash, self.current_cash)
            # 최대 낙폭 계산
            dd = (self.highest_cash - self.current_cash) / self.highest_cash * 100
            self.mdd = max(self.mdd, dd)
        print('='*40)
        print('테스트 결과')
        print('-'*40)
        print('총 거래 횟수 : %s' %self.trade_count)
        print('승리 횟수 : %s' %self.win_count)
        print('승률 : %s' %(self.win_count / self.trade_count * 100))
        print('누적 수익률 : %s' %self.accumulated_ror)
        print('현재 잔액 : %s' % self.current_cash)
        print('최고 잔액 : %s' % self.highest_cash)
        print('최저 잔액 : %s' % self.lowest_cash)
        print('최대 낙폭 (MDD) : %s' % self.mdd)
        print('='*40)

        
df  = pdr.get_data_yahoo('MSFT', '2018-01-01', '2021-12-31')
df = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']]
df.columns = ['open', 'high', 'low', 'close', 'volume']

backtest = MyBackTesting(df, 1000000)
backtest.my_execute()

[*********************100%***********************]  1 of 1 completed
테스트 결과
----------------------------------------
총 거래 횟수 : 436
승리 횟수 : 7
승률 : 1.6055045871559634
누적 수익률 : 2.8493461437318906e-07
현재 잔액 : 0.2849346143731889
최고 잔액 : 1000000
최저 잔액 : 0.2849346143731889
최대 낙폭 (MDD) : 99.99997150653857


In [10]:
import pyupbit
import numpy as np

def my_backtesting(daily_ohlcv, start_cash) :
    K = 0.5                       # 목표매수가 ( 시가 + 변동폭 * K )
    daily_ohlcv = daily_ohlcv     # 일봉 데이터
    fee = 0.0011                  # 수수료 ( calculate_fee.xlsx 참고 )
    buy_signal = False            # 매수 신호

    start_cash   = start_cash     # 시작 자산
    current_cash = start_cash     # 현재 자산
    highest_cash = start_cash     # 자산 최고점
    lowest_cash  = start_cash     # 자산 최저점

    ror = 1                       # 수익률
    accumulated_ror = 1           # 누적 수익률
    mdd = 0                       # 최대 낙폭

    trade_count = 0               # 거래횟수
    win_count = 0                 # 승리횟수

    # 변동폭 ( 고가 - 저가 )
    daily_ohlcv['range'] = daily_ohlcv['high'] - daily_ohlcv['low']
    # 목표매수가 ( 시가 + 변동폭 * K )
    daily_ohlcv['targetPrice'] = daily_ohlcv['open'] + daily_ohlcv['range'].shift(1) * K

    for idx, row in daily_ohlcv.iterrows() :
        # 매수 신호 확인
        # 목표가에 달성한 경우 목표가에 매수해 다음날 시가에 매도한 것으로 판단
        buy_signal = np.where(row['low'] <= row['targetPrice'] <= row['high'], True, False) 
        # 거래횟수 계산
        trade_count += 1 if buy_signal else 0
        # 수익률 계산
        ror = row['close'] / row['targetPrice'] - fee if buy_signal else 1 # 다음날 시가와 당일 종가의 시간차이가 거의 없으므로 종가로 계산
        # 승리 횟수 계산
        win_count += 1 if ror > 1 else 0
        # 누적 수익률 계산
        accumulated_ror *= ror
        # 현재 자산 갱신
        current_cash *= ror
        # 자산 최고점 갱신
        highest_cash = max(highest_cash, current_cash)
        # 자산 최저점 갱신
        lowest_cash = min(lowest_cash, current_cash)
        # 최대 낙폭 계산
        dd = (highest_cash - current_cash) / highest_cash * 100
        mdd = max(mdd, dd)
    print('='*40)
    print('테스트 결과')
    print('-'*40)
    print('총 거래 횟수 : %s' %trade_count)
    print('승리 횟수 : %s' %win_count)
    print('승률 : %s' %(win_count / trade_count * 100))
    print('누적 수익률 : %s' %accumulated_ror)
    print('현재 잔액 : %s' % current_cash)
    print('최고 잔액 : %s' % highest_cash)
    print('최저 잔액 : %s' % lowest_cash)
    print('최대 낙폭 (MDD) : %s' % mdd)
    print('='*40)

        
df  = pdr.get_data_yahoo('MSFT', '2018-01-01', '2021-12-31')
df = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']]
df.columns = ['open', 'high', 'low', 'close', 'volume']

my_backtesting(df, 1000000)

[*********************100%***********************]  1 of 1 completed
테스트 결과
----------------------------------------
총 거래 횟수 : 436
승리 횟수 : 7
승률 : 1.6055045871559634
누적 수익률 : 2.8493428285657543e-07
현재 잔액 : 0.2849342828565761
최고 잔액 : 1000000
최저 잔액 : 0.2849342828565761
최대 낙폭 (MDD) : 99.99997150657171


# Finterstella 
* ref : https://pypi.org/project/finterstellar


In [11]:
import finterstellar2 as fs
import pandas as pd

In [12]:
fs

<module 'finterstellar2' from 'C:\\AI\\pythonProject\\venv\\finterstellar2\\__init__.py'>

In [13]:
K10_dict = {
    '005930' : '삼성전자',
    '000660' : 'SK하이닉스',
    '051910' : 'LG화학',
    '068270' : '셀트리온',
    '005380' : '현대차',
    '207940' : '삼성바이오로직스',
    '005490' : 'POSCO',
    '028260' : '삼성물산',
    '015760' : '한국전력',
    '035420' : 'NAVER'
}
path = './datas/'     # 파일위치
n = 'K10_info.xlsx'     # 파일명
file_name = path + n    # 읽어올 파일위치+파일명 지정
stock_info = pd.read_excel(file_name, converters={'code':str})
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
stock_info.set_index('code', inplace=True)
stock_info.head(10)

name  outstanding
code                         
005930      삼성전자   5969782550
000660    SK하이닉스    728002365
051910      LG화학     70592343
068270      셀트리온    127953489
005380       현대차    213668187
207940  삼성바이오로직스     66165000
005490     POSCO     87186835
028260      삼성물산    189690043
015760      한국전력    641964077
035420     NAVER    164813395

In [14]:
n = 'K10_prices.xlsx'     # 파일명
file_name = path + n    # 읽어올 파일위치+파일명 지정
price = pd.read_excel(file_name)
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
price.set_index('trade_date', inplace=True)
price.tail()

005930  000660  051910  068270  005380    207940  005490  \
trade_date                                                             
2019-02-13   46200   76200  378000  212000  124500  379500.0  267000   
2019-02-14   47500   77400  380000  210000  121500  382000.0  266500   
2019-02-15   46050   73800  380000  209500  121000  380000.0  265500   
2019-02-18   46200   74700  391000  212000  120500  379000.0  262500   
2019-02-19   45950   73700  390000  215000  119500  384500.0  268500   

              028260  015760  035420  
trade_date                            
2019-02-13  117500.0   34000  125000  
2019-02-14  117500.0   33800  127500  
2019-02-15  116000.0   33500  127500  
2019-02-18  116500.0   33050  127000  
2019-02-19  116000.0   33000  126500

In [15]:
import finterstellar2

In [18]:
ld = fs.LoadData()
price = ld.date_formatting(price)
price.tail()

005930  000660  051910  068270  005380    207940  005490  \
Date                                                                   
2019-02-13   46200   76200  378000  212000  124500  379500.0  267000   
2019-02-14   47500   77400  380000  210000  121500  382000.0  266500   
2019-02-15   46050   73800  380000  209500  121000  380000.0  265500   
2019-02-18   46200   74700  391000  212000  120500  379000.0  262500   
2019-02-19   45950   73700  390000  215000  119500  384500.0  268500   

              028260  015760  035420  
Date                                  
2019-02-13  117500.0   34000  125000  
2019-02-14  117500.0   33800  127500  
2019-02-15  116000.0   33500  127500  
2019-02-18  116500.0   33050  127000  
2019-02-19  116000.0   33000  126500

In [19]:
base_date = '2018-01-02'
price = price[base_date:]
# 1월1일에 들어있는 NaN 제거
price.head()


005930  000660  051910  068270  005380    207940  005490  \
Date                                                                   
2018-01-02   51020   76600  411000  225900  149500  389500.0  339000   
2018-01-03   51620   77700  409500  246700  150500  391000.0  357500   
2018-01-04   51080   77100  395500  250000  146500  383000.0  367500   
2018-01-05   52120   79300  405000  266900  149000  390000.0  368000   
2018-01-08   52020   78200  424500  302500  151000  399500.0  369500   

              028260  015760  035420  
Date                                  
2018-01-02  127500.0   37750  177251  
2018-01-03  126500.0   37550  174447  
2018-01-04  128000.0   37200  178854  
2018-01-05  130000.0   37200  181858  
2018-01-08  131000.0   37300  190270

In [20]:
stock_info['outstanding']

code
005930    5969782550
000660     728002365
051910      70592343
068270     127953489
005380     213668187
207940      66165000
005490      87186835
028260     189690043
015760     641964077
035420     164813395
Name: outstanding, dtype: int64

In [21]:
market_cap = pd.DataFrame()
market_cap = price * stock_info['outstanding']
market_cap.head()

005930          000660          051910          068270  \
Date                                                                          
2018-01-02  304578305701000  55764981159000  29013452973000  28904693165100   
2018-01-03  308160175231000  56565783760500  28907564458500  31566125736300   
2018-01-04  304936492654000  56128982341500  27919271656500  31988372250000   
2018-01-05  311145066506000  57730587544500  28589898915000  34150786214100   
2018-01-08  310548088251000  56929784943000  29966449603500  38705930422500   

                    005380        207940          005490        028260  \
Date                                                                     
2018-01-02  31943393956500  2.577127e+13  29556337065000  2.418548e+13   
2018-01-03  32157062143500  2.587052e+13  31169293512500  2.399579e+13   
2018-01-04  31302389395500  2.534120e+13  32041161862500  2.428033e+13   
2018-01-05  31836559863000  2.580435e+13  32084755280000  2.465971e+13   
2018-01-08  32263896237000  2.643292e+13  32215535532500  2.484940e+13   

                    015760          035420  
Date                                        
2018-01-02  24234143906750  29213339077145  
2018-01-03  24105751091350  28751202317565  
2018-01-04  23881063664400  29477534949330  
2018-01-05  23881063664400  29972634387910  
2018-01-08  23945260072100  31359044666650

In [22]:

market_cap['K10'] = market_cap.sum(axis=1)
market_cap.tail()

005930          000660          051910          068270  \
Date                                                                          
2019-02-13  275803953810000  55473780213000  26683905654000  27126139668000   
2019-02-14  283564671125000  56347383051000  26825090340000  26870232690000   
2019-02-15  274908486427500  53726574537000  26825090340000  26806255945500   
2019-02-18  275803953810000  54381776665500  27601606113000  27126139668000   
2019-02-19  274311508172500  53653774300500  27531013770000  27510000135000   

                    005380        207940          005490        028260  \
Date                                                                     
2019-02-13  26601689281500  2.510962e+13  23278884945000  2.228858e+13   
2019-02-14  25960684720500  2.527503e+13  23235291527500  2.228858e+13   
2019-02-15  25853850627000  2.514270e+13  23148104692500  2.200404e+13   
2019-02-18  25747016533500  2.507654e+13  22886544187500  2.209889e+13   
2019-02-19  25533348346500  2.544044e+13  23409665197500  2.200404e+13   

                    015760          035420           K10  
Date                                                      
2019-02-13  21826778618000  20601674375000  5.247950e+14  
2019-02-14  21698385802600  21013707862500  5.330791e+14  
2019-02-15  21505796579500  21013707862500  5.209346e+14  
2019-02-18  21216912744850  20931301165000  5.228707e+14  
2019-02-19  21184814541000  20848894467500  5.214275e+14

In [23]:
index = pd.DataFrame()
# 첫 날짜를 기준으로 지수화
index['K10'] = round( market_cap['K10'] / market_cap.loc[base_date, 'K10'], 4 ) * 100
# 지수 = 일자별 시가총액 / 기준일 시가총액 * 100 
index.head()

K10
Date              
2018-01-02  100.00
2018-01-03  101.39
2018-01-04  100.71
2018-01-05  102.86
2018-01-08  104.12

In [27]:
v = fs.Visualize()

In [28]:
v.price_view(index, base_date, ['K10'])
# price_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 챠트 그리기

In [26]:
K200 = ld.make_historical_price_df(path, 'KOSPI 200')
K200.head()

AttributeError: module 'finterstellar' has no attribute 'str_list'

In [ ]:
index['K200'] = K200.loc[base_date:]
# 1월2일 이후 지수만 idx의 K200 컬럼에 담음
index.head()

In [ ]:
index = index.fillna(method='ffill')
# NaN 값을 앞의 값으로 채움

In [ ]:
v.index_view(index, base_date, ['K200', 'K10'])
# index_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 비교를 위해 인덱스화한 그래프 그리기

In [ ]:
# ! pip install finterstellar

In [ ]:
import finterstellar as fs

symbol = 'MSFT'
df = fs.get_price(symbol, start_date='2020-01-01', end_date='2020-12-31')
fs.draw_chart(df, right=symbol)
fs.macd(df)

In [ ]:
df.head()

In [ ]:
fs.draw_chart(df, left='ema_short', right='macd_signal')

In [ ]:
df.head()

In [ ]:
df = fs.backtest(signal=signal, data=df, m=2, cost=.001)

In [ ]:
df.rename(columns={"MSFT":"Price"}, inplace=True)
fs.draw_price_multiple_band(df, multiple='ema_short', acct='macd_signal')

* http://data.krx.co.kr/contents/MDC/MGZN/publication/MDCMGZN001.cmd?menuId=MDCMGZN015

In [ ]:
from pykrx import stock 
# idx_list = stock.get_market_cap("20230403")[:10].index
시총s = stock.get_market_cap("20230403")[:10]["시가총액"]
#print(시총s.index, 시총s.values)

ticker_list = []
for ticker, 시총 in 시총s.items(): 
    티커명 = stock.get_market_ticker_name(ticker)
    ticker_list.append((ticker, 티커명, 시총))
시총df = pd.DataFrame(ticker_list, columns=['code','name','marketcap'] )
시총df

## Kospi 10 종목코드와 가중치 데이터

In [ ]:
print(np.sum(시총df['marketcap'] / 시총df['marketcap'].sum()))
시총df['marketcap_rate'] = 시총df['marketcap'] / 시총df['marketcap'].sum()
시총df

In [ ]:
# 종목 가격 데이터 불러오기
price_df = pd.read_csv('./datasets/kospi_top10.csv', index_col='날짜')
# import FinanceDataReader as fdr
# price_df = fdr.DataReader('KS11', '2022')
# 일별 수익률 계산
return_df = price_df.pct_change()
return_df.head()

In [ ]:
return_df.dropna(axis=0, inplace=True)
return_df

In [ ]:
return_df.sum(axis=0) 

In [ ]:
 시총df['marketcap_rate']

In [ ]:
시총df['marketcap_rate']

In [ ]:
시총df["return_rate"]   = return_df.sum(axis=0).values
시총df

In [ ]:
market_cap_df = return_df.values * 시총df['marketcap_rate'].values
cap_df = pd.DataFrame(market_cap_df, columns=return_df.columns, index=return_df.index)
cap_df['K10'] = cap_df.sum(axis=1)
cap_df

In [ ]:
index = pd.DataFrame()
# 첫 날짜를 기준으로 지수화
index['K10'] = round( cap_df['K10'] / cap_df.loc['2023-04-04', 'K10'], 4 ) * 100
# 지수 = 일자별 시가총액 / 기준일 시가총액 * 100 
index.tail()

In [ ]:
index.plot()

In [ ]:

K200 = ld.make_historical_price_df(path, 'KOSPI 200')

# 슬랙 봇
* ref : https://hyojun.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%BD%94%EC%9D%B8-%EC%9E%90%EB%8F%99%EB%A7%A4%EB%A7%A4-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8-%EB%A7%8C%EB%93%A4%EA%B8%B0-5-%EC%8A%AC%EB%9E%99-%EC%95%8C%EB%A6%BC-%EB%B4%87?category=1046904

In [ ]:
# 에러 ------------ 사용불가 
# backtesting.set_bokeh_output(notebook=False)

In [ ]:
# #Strategy 작성
# from backtesting import Strategy
# from backtesting.lib import crossover

# class MyStrategy(Strategy):
#     n1 = 10
#     n2 = 30
#     def init(self):
#         self.sma1 = self.I(SMA, self.data.Close, self.n1)
#         self.sma2 = self.I(SMA, self.data.Close, self.n2)

#     def next(self):
        
#         if not self.position:          # not in the market
#             if self.crossover > 0:     # if fast crosses slow to the upside
#                 self.buy()             # enter long
#         elif self.crossover < 0:       # in the market & cross to the downside
#             self.close()               # close long position

# from backtesting import Backtest
# data  = pdr.get_data_yahoo('MSFT', '2018-01-01', '2021-12-31')
# bt = Backtest(data, MyStrategy, cash=100000, commission=.002)

# #통계 수치들을 보고싶다면
# stats = bt.run()
# stats
# #시각적으로 나타내려면
# bt.plot()